In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

# 计算lightGBM+BiLSTM
df_lightGBM = pd.read_csv("./result/lightGBMpre.csv")
df_BiLSTM = pd.read_csv("./result/BiLSTMpre.csv")

In [2]:
# 计算基模型的累计误差
def CumMSE(df_model:pd.DataFrame):
    """
    df_model第一列为true，第二列为pre
    """
    cum_mse = []
    for i in range(len(df_model)):
        cum_mse.append(mean_squared_error(df_model.iloc[:i+1,0],df_model.iloc[:i+1,1]))
    return cum_mse

cum_mse_lightGBM = CumMSE(df_lightGBM)
cum_mse_BiLSTM = CumMSE(df_BiLSTM)

# 计算组合权重
def CombineWeight(cum_mse1:list,cum_mse2:list):
    """
    :parameters:
    cum_mse1: 模型1累计误差 
    cum_mse2: 模型2累计误差
    
    :return: 
    模型1的组合权重
    """
    weightList = []
    for i in range(len(cum_mse1)):
        weightOfModel1 = cum_mse1[i]/sum([cum_mse1[i],cum_mse2[i]])
        weightList.append(weightOfModel1)
    return weightList

weight_lightGBM = CombineWeight(cum_mse_lightGBM,cum_mse_BiLSTM)
    

In [3]:
# 计算lightGBM+BiLSTM组合预测值
def CombineModel(df_Model1:pd.DataFrame,df_Model2:pd.DataFrame,combineWeightOfModel1:list):
    """
    :parameters:
    df_Model1: 模型1的结果
    df_Model2: 模型2的结果
    combineWeightOfModel1: 模型1的组合权重

    :return: 预测值
    """
    y_pre = []
    for idx,weight in enumerate(combineWeightOfModel1):
        prediction = df_Model1['y_pre'][idx] *weight + df_Model2['y_pre'][idx]*(1-weight)
        y_pre.append(prediction)
    return y_pre

y_pre = CombineModel(df_lightGBM,df_BiLSTM,weight_lightGBM)
y = df_lightGBM.loc[:,'y_true']


In [4]:
# 计算lightGBM+BiLSTM组合评价
r2Score = r2_score(y_true=y, y_pred=y_pre)
meanSquaredError = mean_squared_error(y_true=y, y_pred=y_pre)
meanAbsoluteError = mean_absolute_error(y_true=y, y_pred=y_pre)
print("r2Score: ", r2Score)
print("meanSquaredError: ", meanSquaredError)
print('RMSE: ',np.sqrt(meanSquaredError))
print("meanAbsoluteError: ", meanAbsoluteError)


r2Score:  0.9751975962932983
meanSquaredError:  0.0012807372445758141
RMSE:  0.03578738946299121
meanAbsoluteError:  0.023601589518249633
